# Cell 1: Install necessary packages in Jupyter

In [ ]:
#!pip install networkx pygraphviz pydot

# graphviz installation may require some additional steps:
# brew install graphviz
# export PKG_CONFIG_PATH="/opt/homebrew/lib/pkgconfig"
# CFLAGS="-I$(brew --prefix graphviz)/include" LDFLAGS="-L$(brew --prefix graphviz)/lib" pip install pygraphviz

# Imports and function definitions

In [36]:
import ast
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
from io import StringIO
import re

# Impressive ASCII header and footer markers as global variables
GRAPH_HEADER = (
    "# ╔══════════════════════════════════════════════════════════════╗\n"
    "# ║                      ★ DEPENDENCY GRAPH ★                    ║\n"
    "# ║                     BEGIN DEPENDENCY GRAPH                   ║\n"
    "# ╚══════════════════════════════════════════════════════════════╝"
)
GRAPH_FOOTER = (
    "# ╔══════════════════════════════════════════════════════════════╗\n"
    "# ║                     END DEPENDENCY GRAPH                     ║\n"
    "# ╚══════════════════════════════════════════════════════════════╝"
)

# Helper function to parse the Python file and extract dependencies


def parse_file(filename):
    with open(filename, "r") as file:
        tree = ast.parse(file.read())
    return tree


# Helper function to check if a node is top-level (not within a class)
def is_top_level(node, tree):
    return not any(isinstance(parent, ast.ClassDef) and hasattr(parent, 'body') and node in parent.body for parent in ast.walk(tree))


# Function to analyze structure and dependencies
def analyze_structure_and_dependencies(filename):
    with open(filename, "r") as file:
        tree = ast.parse(file.read())

    dependencies = nx.DiGraph()
    class_methods = {}
    standalone_functions = []

    for node in tree.body:
        if isinstance(node, ast.ClassDef):
            current_class = node.name
            class_methods[current_class] = []

            for class_node in node.body:
                if isinstance(class_node, ast.FunctionDef):
                    method_name = f"{current_class}.{class_node.name}"
                    class_methods[current_class].append(method_name)
                    dependencies.add_node(method_name)

                    for inner_node in ast.walk(class_node):
                        if isinstance(inner_node, ast.Call) and isinstance(inner_node.func, ast.Attribute):
                            called_method = f"{current_class}.{inner_node.func.attr}"
                            if called_method in dependencies:
                                dependencies.add_edge(
                                    method_name, called_method)

        elif isinstance(node, ast.FunctionDef) and is_top_level(node, tree):
            function_name = node.name
            standalone_functions.append(function_name)
            dependencies.add_node(function_name)

            for inner_node in ast.walk(node):
                if isinstance(inner_node, ast.Call) and isinstance(inner_node.func, ast.Name):
                    called_function = inner_node.func.id
                    if called_function in standalone_functions:
                        dependencies.add_edge(function_name, called_function)

    return {
        "dependencies": dependencies,
        "class_methods": class_methods,
        "standalone_functions": standalone_functions
    }


# Function to generate ASCII graph with commented lines
def generate_condensed_ascii_graph(analysis_result):
    ascii_output = StringIO()
    dependencies = analysis_result["dependencies"]
    class_methods = analysis_result["class_methods"]
    standalone_functions = analysis_result["standalone_functions"]

    for class_name, methods in class_methods.items():
        ascii_output.write(f"\n# {'='*10} [ Class: {class_name} ] {'='*10}\n")

        for method in methods:
            call_count = dependencies.in_degree(method)
            ascii_output.write(f"# {method} ( <- {call_count} x)\n")

            for called_method in dependencies.successors(method):
                if called_method in methods:
                    ascii_output.write(f"#   -> {called_method}\n")

    if standalone_functions:
        ascii_output.write(f"\n# {'='*10} [ Non-Class Functions ] {'='*10}\n")

        for function in standalone_functions:
            call_count = dependencies.in_degree(function)
            ascii_output.write(f"# {function} ( <- {call_count} x)\n")

            for called_function in dependencies.successors(function):
                if called_function in standalone_functions:
                    ascii_output.write(f"#   -> {called_function}\n")

    ascii_art = ascii_output.getvalue()
    ascii_output.close()
    return ascii_art


# Function to check if the file already contains the graph block
def get_existing_graph(filename):
    with open(filename, "r") as file:
        content = file.read()
    match = re.search(
        re.escape(GRAPH_HEADER) + r"(.+?)" + re.escape(GRAPH_FOOTER),
        content, re.DOTALL
    )
    return match.group(0) if match else None


# Function to update the Python file with the ASCII graph
def update_file_with_graph(filename, ascii_art):
    with open(filename, "r") as file:
        content = file.read()

    # Define the fully commented graph section
    graph_section = f"{GRAPH_HEADER}\n{ascii_art}\n{GRAPH_FOOTER}"

    # Ensure all lines within the existing graph are commented
    existing_graph = get_existing_graph(filename)
    if existing_graph:
        for line in existing_graph.splitlines():
            if line.strip() and not line.strip().startswith("#"):
                raise ValueError(
                    "Error: Un-commented lines found within the existing graph section.")

    # Replace existing or add new graph
    if existing_graph:
        new_content = re.sub(
            re.escape(GRAPH_HEADER) + r"(.+?)" + re.escape(GRAPH_FOOTER),
            graph_section, content, flags=re.DOTALL
        )
    else:
        new_content = f"{graph_section}\n\n{content}"

    # Write the updated content back to the file
    with open(filename, "w") as file:
        file.write(new_content)

    print("File updated with new dependency graph.")


# Function to check and update the graph in the file if needed
def check_and_update_graph(filename):
    analysis_result = analyze_structure_and_dependencies(filename)
    new_ascii_art = generate_condensed_ascii_graph(analysis_result)

    existing_graph = get_existing_graph(filename)
    new_graph_section = f"{GRAPH_HEADER}\n{new_ascii_art}\n{GRAPH_FOOTER}"

    if not existing_graph or existing_graph.strip() != new_graph_section.strip():
        update_file_with_graph(filename, new_ascii_art)
        print("File was updated with the new dependency graph.")
    else:
        print("File is up to date; no changes needed.")

# Display ASCII graph in Jupyter

In [ ]:
filename = "../hrmlib/hrmtools.py"
display_ascii_graph(filename)

# Cell 4: Update the Python file with the new ASCII graph if it has changed

In [41]:
# Set the target filename
filename = "../hrmlib/hrmtools.py"

# Call the function to check and update the ASCII graph in the file
check_and_update_graph(filename)

File is up to date; no changes needed.


# Cell 5: Full check-update function